### This code downloads XML bulks of patent FULL TEXT data from USPTO, parses it into a dataframe and creates a flag for patents supporting working-from-home technologies.

In [2]:
#Uploading necessary packages
import pandas as pd
import pandas_read_xml as pdx
import numpy as np
import re
import os

In [ ]:
for j in range(2010,2021):
    print("Processing year {}".format(j))
    
    directory = '/Users/yulia_zhestkova/Patent parsing/full text/'+str(j)+'/'
    os.chdir('/Users/yulia_zhestkova/Patent parsing/full text/'+str(j)+'/')


    i=0
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file=open(file=filename, mode='r')
            file_content_raw=file.read()
            file.close()
            text1=re.compile("<\?xml version=\"1\.0\" encoding\=\"UTF\-8\"\?>")
            file_content=text1.split(file_content_raw)
            while '' in file_content:
                file_content.remove('')
        
            print("Processing file {}".format(filename)) 
            df=pd.DataFrame(file_content, columns=['application_full'])
            df['application_full'].replace(to_replace=r'\n', value='', regex=True, inplace=True)
            df['application_full'].replace(to_replace=r'\"', value='', regex=True, inplace=True)
            df['info']=df['application_full'].apply(lambda x: "" if re.search(r'<application-reference(.*)</application-reference>', x)==None else re.search(r'<application-reference(.*)</application-reference>', x).group(1))
            df['app_id']=df['info'].apply(lambda x: "" if re.search(r'<doc-number>(.*)</doc-number>', x)==None else re.search(r'<doc-number>(.*)</doc-number>', x).group(1))
            df['date']=df['info'].apply(lambda x: "" if re.search(r'<date>(.*)</date>', x)==None else re.search(r'<date>(.*)</date>', x).group(1))
            df['description']=df['application_full'].apply(lambda x: "" if re.search(r'<\?summary-of-invention description=Summary of Invention end=lead\?>(.*)<\?summary-of-invention description=Summary of Invention end=tail\?>', x)==None else re.search(r'<\?summary-of-invention description=Summary of Invention end=lead\?>(.*)<\?summary-of-invention description=Summary of Invention end=tail\?>', x).group(1))
            df['description']=df['description'].apply(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', x))
            df=df.drop(['application_full','info'], axis=1)
            df['description']=df['description'].str.lower()

            if i==0:
                master_df=df
            else:
                master_df=master_df.append(df, ignore_index=True)
    
            i=i+1
            del df

    master_df.to_csv(r'completetext'+str(j)+'.csv', index = None)
    wfh=['telecommuting', 'telework', 'teleworking', 'working from home', 'mobile work', 'remote work', 'flexible workplace', 'work from home', 'mobile working', 'remote working', 'work remotely', 'working remotely', 'remote workplace', 'telecommuter', 'teleworker', 'home-sourced worker', 'home-sourced employee', 'work-at-home', 'work at home', 'telecommuting specialist', 'nomadic worker', 'nomadic employee', 'work-from-home', 'work-from-anywhere', 'video conference', 'video conferencing', 'virtual office', 'distance work', 'flexible work', 'virtual work', 'virtual office', 'virtual employee', 'home office', 'home-based office', 'home-based work', 'work from anywhere', 'working from anywhere', 'work-from-anywhere', 'digital workplace', 'video chat', 'video call', 'teleconference', 'teleconferencing', 'working from a remote location', 'work from a remote location']


    master_df['description']=master_df['description'].replace({w: "wfhflag" for w in wfh}, regex=True)
    master_df['count flags']=master_df['description'].str.count("wfhflag")
    master_df=master_df[['date', 'app_id', 'count flags']]
    
    master_df.to_csv (r'wfhfflags'+str(j)+'.csv', index = None)

In [ ]:
#Redefining files directory
directory = '/Users/yulia_zhestkova/Patent parsing/full text/'
os.chdir('/Users/yulia_zhestkova/Patent parsing/full text/')

#Processing complete csv files (output of the previous code) 
for j in range(2010,2021):
    print("Processing file {}".format(j))
    
    df = pd.read_csv(str(j)+'/wfhfflags'+str(j)+'.csv')    
    if j==2010:
        df_master=df
    else:
        df_master=df_master.append(df, ignore_index=True)
        
df_master.to_csv (r'wfhflag.csv', index = None)